In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import cv2
import sklearn
import pickle
%matplotlib inline

### Data pre processing

In [2]:
xy_tup = []
graph_dat = []

def data_sum(name):
    #reading the data
    col_names = ['Ctr','Lt','Rt','s_angle','throttle','break','speed']
    df = pd.read_csv('Data/'+name+'_Data/driving_log.csv', header=None, names=col_names)
    #print(len(df))
    #print(df.s_angle.value_counts(bins=20, sort=False))
    
    for i in range(len(df.index)):
        angle = df['s_angle'][i]
        
        if name == 'Sam' or angle != 0:
            #update to new address
            source_path = df['Ctr'][i]
            filename = source_path.split('/')[-1]
            current_path = 'Data/'+name+'_Data/IMG/'+filename #New_path = '../data/IMG/'

            image = cv2.imread(current_path)
            tuple_temp = (image,angle)
            xy_tup.append(tuple_temp)
            graph_dat.append(angle)
            if angle !=0:
                tuple_temp_inv = (np.fliplr(image),-angle)
                xy_tup.append(tuple_temp_inv)
                graph_dat.append(-angle)


names = ['Sam','Nor','Rev','Tra']
for nam in names:
    print(nam)
    data_sum(nam)

'''
#plotting histogram of steering angles
    sns.distplot(df.s_angle, bins=20,kde=False, color='green')
    print(df.s_angle.value_counts(bins=20, sort=False))
'''

Sam
Nor
Rev
Tra


"\n#plotting histogram of steering angles\n    sns.distplot(df.s_angle, bins=20,kde=False, color='green')\n    print(df.s_angle.value_counts(bins=20, sort=False))\n"

In [12]:
#sns.distplot(graph_dat, bins=20,kde=False, color='green')
print(xy_tup[0][1])

0.0


In [13]:
xy_tup = np.array(xy_tup)

with open('final_data.pickle', 'wb') as filef:
    pickle.dump(xy_tup, filef)

### Testing traing split

In [5]:
from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(xy_tup, test_size=0.2)

### Generator for data

In [6]:
def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                center_image = batch_sample[0]
                center_angle = batch_sample[1]
                images.append(center_image)
                angles.append(center_angle)
                
            X_train = np.array(images) 
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)
            
# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)

In [7]:
import keras

from keras.models import Sequential
from keras.layers import Lambda, Cropping2D
from keras.layers.core import Dense, Activation, Flatten, Dropout

from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

Using TensorFlow backend.


### Architecture

In [8]:
model = Sequential()

'''50 rows pixels from the top of the image, 20 rows pixels from the bottom of the image,0 columns of pixels
from the left of the image 0 columns of pixels from the right of the image are cropped'''
model.add(Cropping2D(cropping=((50,20), (0,0)), input_shape=(160,320,3)))

#Adding lambda layer that normalizes and mean centers the image
model.add(Lambda(lambda x: (x / 255.0) - 0.5))

#convolutional layers
"Layer-1"
model.add(Convolution2D(32,5,5, subsample=(2,2), border_mode='valid')) #subsample means stride
#2x2 max-pooling layer
model.add(MaxPooling2D((2, 2))) #by default it is a valid padding
model.add(Activation('relu'))
#Dropout layer
model.add(Dropout(0.5))

"Layer-2"
model.add(Convolution2D(64,3,3, subsample=(2,2), border_mode='valid'))
#2x2 max-pooling layer
model.add(MaxPooling2D((2, 2)))
model.add(Activation('relu'))
#Dropout layer
model.add(Dropout(0.5))

"Layer-3"
model.add(Convolution2D(96,2,2, border_mode='valid')) #stride by default is (1,1)
#2x2 max-pooling layer
model.add(MaxPooling2D((2, 2)))
model.add(Activation('relu'))
#Dropout layer
model.add(Dropout(0.5))

#flattening the layer
model.add(Flatten())

"Vanilla neural network"
model.add(Dense(610))
model.add(Activation('relu'))

model.add(Dense(377))
model.add(Activation('relu'))

model.add(Dense(233))
model.add(Activation('relu'))

model.add(Dense(144))
model.add(Activation('relu'))

model.add(Dense(89))
model.add(Activation('relu'))

model.add(Dense(55))
model.add(Activation('relu'))

model.add(Dense(34))
model.add(Activation('relu'))

model.add(Dense(21))
model.add(Activation('relu'))

model.add(Dense(13))
model.add(Activation('relu'))

model.add(Dense(8))
model.add(Activation('relu'))

model.add(Dense(1))

model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

#model.fit(X,Y, batch_size=100, epochs=1, validation_split=0.2, verbose=1)

hist_obj = model.fit_generator(train_generator, samples_per_epoch= len(train_samples), 
                    /validation_data=validation_generator,nb_val_samples=len(validation_samples), 
                     /nb_epoch=3, verbose=1)

model.save(model.h5)